In [7]:
import pandas as pd
import numpy as np

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

2023-01-10 13:34:35.147018: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
# data = pd.read_csv('../Data/Sequencing/sequenced_data.csv')
data = pd.read_csv('../Data/Processed Data/complete_data.csv')
data = data.copy()
data

,Gene Name,Solubility(%),Yield(uM),Yield(ug/ml),Minus Sol,TF Sol,GroE Sol,KJE Sol,Minus(uM),TF(uM),...,Gene product description,Cell location,Structure (PDB) id,SCOP assignment,Sequence,Sequence length,Sequence mass,Organism,UP_ID,UP_NAME
0,thrB,32.0,2.3,78.0,NaN,NaN,NaN,NaN,NaN,NaN,...,homoserine kinase,Cytoplasmic,NaN,54211; Ribosomal protein S5 domain 2-like 5506...,MVKVYAPASSANMSVGFDVLGAAVTPVDGALLGDVVTVEAAETFSL...,310,33624,Escherichia coli (strain K12 / DH10B),B1XBC8,KHSE_ECODH
1,thrC,18.0,1.7,78.0,46.53,45.80,83.44,63.61,0.863,0.761,...,threonine synthase,Cytoplasmic,NaN,53686; Tryptophan synthase beta subunit-like P...,MKLYNLKDHNEQVSFAQAVTQGLGKNQGLFFPHDLPEFSLTEIDEM...,428,47114,Escherichia coli (strain K12),"['P00934', 'Q6LEK8']",THRC_ECOLI
2,yaaX,78.0,1.2,14.0,NaN,NaN,NaN,NaN,NaN,NaN,...,predicted protein,Periplasmic,NaN,NaN,MKKMQSIVLALSLVLVAPMAAQAAEITLVPSVKLQIGDRDNRGYYW...,98,11355,Escherichia coli (strain K12),P75616,YAAX_ECOLI
3,yaaA,7.0,2.4,71.0,14.01,26.65,52.96,17.69,1.199,1.014,...,conserved protein,Cytoplasmic,NaN,NaN,MLILISPAKTLDYQSPLTTTRYTLPELLDNSQQLIHEARKLTPPQI...,258,29586,Escherichia coli (strain K12 / DH10B),B1XBD1,YAAA_ECODH
4,talB,85.0,2.7,94.0,NaN,NaN,NaN,NaN,NaN,NaN,...,transaldolase B,Cytoplasmic,1ONR,51569; Aldolase,MTDKLTSLRQYTTVVADTGDIAAMKLYQPQDATTNPSLILNAAQIP...,317,35219,Escherichia coli (strain K12),"['P0A870', 'P30148']",TALB_ECOLI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3155,ytfP,98.0,0.9,12.0,NaN,NaN,NaN,NaN,NaN,NaN,...,conserved protein,Cytoplasmic,NaN,NaN,MRIFVYGSLRHKQGNSHWMTNAQLLGDFSIDNYQLYSLGHYPGAVP...,113,12866,Escherichia coli (strain K12),"['P0AE48', 'P39323', 'Q2M683']",YTFP_ECOLI
3156,yzfA,88.0,0.4,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,predicted protein,Cytoplasmic,NaN,NaN,MHMVTYPCLTSRRFQLALIHRRVVDKRTSMHSRTASESTGARIHRP...,89,10585,Escherichia coli (strain K12),A0A385XK32,A0A385XK32_ECOLI
3157,chpB,94.0,0.4,5.0,NaN,NaN,NaN,NaN,NaN,NaN,...,toxin of the ChpB-ChpS toxin-antitoxin system,Cytoplasmic,NaN,NaN,MRITIKRWGNSAGMVIPNIVMKELNLQPGQSVEAQVSNNQLILTPI...,83,9272,Escherichia coli (strain K12),"['P08365', 'P76803', 'Q2M681']",CHPS_ECOLI
3158,nrdG,69.0,0.4,7.0,NaN,NaN,NaN,NaN,NaN,NaN,...,anaerobic ribonucleotide reductase activating ...,Cytoplasmic,NaN,NaN,MNYHQYYPVDIVNGPGTRCTLFVSGCVHECPGCYNKSTWRVNSGQP...,154,17446,Escherichia coli (strain K12),"['P0A9N8', 'P39329', 'Q2M670']",NRDG_ECOLI


In [52]:
def toNumeric(data):
    char = list(data.unique())
    temp = []
    for x in data:
        temp.append(char.index(x))
    return temp

In [54]:
data['Type of gene product numeric'] = toNumeric(data['Type of gene product'])
data['Cell location numeric'] = toNumeric(data['Cell location'])

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [55]:
X = data[['Yield(uM)','Yield(ug/ml)','Calculated MW(kDa)','Calculated pI', 'Sequence length', 'Sequence mass','Type of gene product numeric', 'Cell location numeric']]
# X = data[['Yield(uM)','Yield(ug/ml)','Calculated MW(kDa)','Calculated pI', 'Sequence length', 'Sequence mass',]]
# X = data[['Yield(uM)','Yield(ug/ml)','Calculated MW(kDa)','Calculated pI', 'Type of gene product Integer']]
y = data['Solubility(%)']

In [56]:
def normalize(col_data):
    return (col_data - col_data.min())/(col_data.max()-col_data.min())

In [ ]:
X = data.iloc[:, 29:2029]

In [57]:
for column in X:
    X[column] = normalize(X[column])
X = round(X,2)

/var/folders/__/8qxqknx91_l7s7jy54cf7p2h0000gn/T/ipykernel_28695/3188529497.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = normalize(X[column])


In [58]:
y = round(normalize(y),2)

In [59]:
X

,Yield(uM),Yield(ug/ml),Calculated MW(kDa),Calculated pI,Sequence length,Sequence mass,Type of gene product numeric,Cell location numeric
0,0.15,0.21,0.21,0.26,0.22,0.22,0.00,0.00
1,0.11,0.21,0.31,0.21,0.31,0.31,0.00,0.00
2,0.08,0.03,0.06,0.76,0.07,0.07,0.05,0.09
3,0.15,0.19,0.18,0.53,0.18,0.19,0.05,0.00
4,0.17,0.25,0.22,0.22,0.23,0.23,0.00,0.00
...,...,...,...,...,...,...,...,...
3155,0.06,0.03,0.07,0.47,0.08,0.08,0.05,0.00
3156,0.03,0.01,0.05,0.85,0.06,0.06,0.05,0.00
3157,0.03,0.01,0.07,0.30,0.06,0.06,0.15,0.00
3158,0.03,0.01,0.10,0.43,0.11,0.11,0.00,0.00


In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [44]:
model = tf.keras.Sequential()

2023-01-10 14:45:46.125425: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
model.add(layers.Dense(3, input_dim=6, activation='sigmoid'))
model.add(layers.Dense(1, activation='relu'))

In [ ]:
# compile the keras model
#model.compile(loss='mean_absolute_error', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
     #metrics=[tf.keras.metrics.MeanSquaredError(name='MSE'), tf.keras.metrics.RootMeanSquaredError(name='MRSE')])

model.compile(loss='mean_absolute_error', optimizer=tf.keras.optimizers.Adam(0.001),  metrics=[tf.keras.metrics.MeanSquaredError(name='MSE'), tf.keras.metrics.RootMeanSquaredError(name='MRSE')])

history = model.fit(
    X_train,
    y_train,
    batch_size=20,
    epochs=200
)

Epoch 1/200


ValueError: in user code:

    File "/shared-libs/python3.9/py/lib/python3.9/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/shared-libs/python3.9/py/lib/python3.9/site-packages/keras/engine/training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/shared-libs/python3.9/py/lib/python3.9/site-packages/keras/engine/training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "/shared-libs/python3.9/py/lib/python3.9/site-packages/keras/engine/training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "/shared-libs/python3.9/py/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/shared-libs/python3.9/py/lib/python3.9/site-packages/keras/engine/input_spec.py", line 277, in assert_input_

In [ ]:
model.save('../models/models_SNN')

INFO:tensorflow:Assets written to: ../models/models_SNN/assets


In [ ]:
eval_results = model.evaluate(X_test, y_test)

20/20 [==============================] - 0s 1ms/step - loss: 0.6184 - MSE: 0.0426 - MRSE: 0.2065


In [ ]:
print("loss, accuracy", eval_results)

loss, accuracy [0.6184118986129761, 0.04262311011552811, 0.20645365118980408]


In [ ]:
print("loss, accuracy", eval_results * 100, '%')

loss, accuracy 4.173898324370384 %


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=8ea8948f-b978-492f-a6dd-8ac5feb4b472' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>